# 🧠 fMRI Cognitive State Classifier Pipeline
This notebook demonstrates end-to-end analysis of the Haxby fMRI dataset using PCA and machine learning classification.

In [ ]:
# Step 1: Imports
from src.preprocess import load_and_preprocess_data
from src.model import BrainNet
from src.train import create_dataloaders, train_model
from src.visualize import plot_confusion, print_classification_report, show_pca_brain_map
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch


In [ ]:
# Step 2: Load and preprocess data
X, y, masker = load_and_preprocess_data()

In [ ]:
# Step 3: Dimensionality Reduction with PCA
pca = PCA(n_components=100)
X_reduced = pca.fit_transform(X)

In [ ]:
# Step 4: Prepare train/test split
le = LabelEncoder()
y_enc = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y_enc, test_size=0.2, random_state=42)

In [ ]:
# Step 5: Train Deep Learning Model
model = BrainNet(input_dim=100, num_classes=len(le.classes_))
train_loader, val_loader = create_dataloaders(X_train, y_train, X_test, y_test)
trained_model = train_model(model, train_loader, val_loader)

In [ ]:
# Step 6: Evaluate Model
trained_model.eval()
y_preds, y_true = [], []
with torch.no_grad():
    for X_batch, y_batch in val_loader:
        outputs = trained_model(X_batch)
        _, preds = torch.max(outputs, 1)
        y_preds.extend(preds.numpy())
        y_true.extend(y_batch.numpy())
plot_confusion(y_true, y_preds, class_labels=list(range(len(le.classes_))))
print_classification_report(y_true, y_preds, le.classes_)

In [ ]:
# Step 7: Visualize a PCA Brain Map
show_pca_brain_map(pca.components_, masker, component_idx=0)